In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta
import os
import csv
import boto3
import pymysql
os.chdir('/Users/sramachandran/Dropbox (Move)/Crime Busters/')

In [2]:
crime_data = pd.read_csv('Police_Department_Incident_Reports__Historical_2003_to_May_2018.csv', low_memory = False)
print(crime_data.shape)
crime_data.head(10)

(145869, 13)


,IncidntNum,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Location,PdId
0,180267742,FRAUD,"FALSE CLAIMS, PRESENTING TO GOVERNMENT",Monday,05/01/2017,00:01,CENTRAL,NONE,200 Block of POST ST,-122.406022,37.788649,"(37.788649124529414, -122.4060218384839)",18026774209173
1,180190529,OTHER OFFENSES,MISCELLANEOUS INVESTIGATION,Monday,05/01/2017,12:00,CENTRAL,NONE,800 Block of GRANT AV,-122.406460,37.794704,"(37.794704150989254, -122.40646032018432)",18019052968020
2,180129396,FRAUD,FRAUDULENT USE OF AUTOMATED TELLER CARD,Monday,05/01/2017,00:01,TENDERLOIN,NONE,500 Block of OFARRELL ST,-122.413798,37.785630,"(37.78562979423735, -122.41379841586897)",18012939609250
3,176166112,LARCENY/THEFT,PETTY THEFT OF PROPERTY,Monday,05/01/2017,00:01,INGLESIDE,NONE,WILDER ST / NATICK ST,-122.431595,37.733766,"(37.73376551140977, -122.43159535136645)",17616611206372
4,176165891,LARCENY/THEFT,GRAND THEFT OF PROPERTY,Monday,05/01/2017,13:30,CENTRAL,NONE,300 Block of SANSOME ST,-122.401201,37.793774,"(37.793774373929594, -122.40120121358764)",17616589106374
5,176140291,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Monday,05/01/2017,22:43,TARAVAL,NONE,200 Block of VICENTE ST,-122.468724,37.739911,"(37.739910543879205, -122.46872393000805)",17614029106244
6,176140003,VANDALISM,"MALICIOUS MISCHIEF, VANDALISM",Monday,05/01/2017,19:00,TENDERLOIN,NONE,100 Block of POWELL ST,-122.408014,37.785951,"(37.78595077676134, -122.40801449292017)",17614000328150
7,176134937,NON-CRIMINAL,LOST PROPERTY,Monday,05/01/2017,11:28,INGLESIDE,NONE,200 Block of ONEIDA AV,-122.443010,37.721870,"(37.72187048254208, -122.44300998676374)",17613493771000
8,176134692,NON-CRIMINAL,LOST PROPERTY,Monday,05/01/2017,12:48,TENDERLOIN,NONE,700 Block of OFARRELL ST,-122.417085,37.785215,"(37.78521535904708, -122.41708535742424)",17613469271000
9,176126592,LARCENY/THEFT,PETTY THEFT OF PROPERTY,Monday,05/01/2017,11:00,NORTHERN,NONE,1600 Block of JACKSON ST,-122.422253,37.794092,"(37.79409172603437, -122.4222532995645)",17612659206372


In [6]:
crime_data.groupby(['Category'], as_index = False).agg({'IncidntNum':'nunique'})\
.sort_values(by = 'IncidntNum',ascending = False)

,Category,IncidntNum
16,LARCENY/THEFT,44717
20,NON-CRIMINAL,16151
21,OTHER OFFENSES,13713
1,ASSAULT,11170
35,VANDALISM,8973
4,BURGLARY,5439
32,SUSPICIOUS OCC,5370
36,VEHICLE THEFT,4631
37,WARRANTS,4483
25,ROBBERY,3096


In [7]:
violent = ['ASSAULT', 'EXTORTION', 'SEX OFFENSES, NON FORCIBLE','SEX OFFENSES, FORCIBLE','KIDNAPPING', \
           'WEAPON LAWS', 'MISSING PERSON', 'ROBBERY', 'BURGLARY']
non_violent = ['LARCENY/THEFT', 'VEHICLE THEFT', 'VANDALISM', 'FRAUD', 'DRUG/NARCOTIC']

In [11]:
crime_data.loc[crime_data['Category'].isin(violent), 'type'] = 1
crime_data.loc[crime_data['Category'].isin(non_violent), 'type'] = 0
crime_data = crime_data.dropna(subset = ['type'])
print(crime_data.shape)
crime_data.head(10)

(94982, 14)


,IncidntNum,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Location,PdId,type
0,180267742,FRAUD,"FALSE CLAIMS, PRESENTING TO GOVERNMENT",Monday,05/01/2017,00:01,CENTRAL,NONE,200 Block of POST ST,-122.406022,37.788649,"(37.788649124529414, -122.4060218384839)",18026774209173,0.0
2,180129396,FRAUD,FRAUDULENT USE OF AUTOMATED TELLER CARD,Monday,05/01/2017,00:01,TENDERLOIN,NONE,500 Block of OFARRELL ST,-122.413798,37.785630,"(37.78562979423735, -122.41379841586897)",18012939609250,0.0
3,176166112,LARCENY/THEFT,PETTY THEFT OF PROPERTY,Monday,05/01/2017,00:01,INGLESIDE,NONE,WILDER ST / NATICK ST,-122.431595,37.733766,"(37.73376551140977, -122.43159535136645)",17616611206372,0.0
4,176165891,LARCENY/THEFT,GRAND THEFT OF PROPERTY,Monday,05/01/2017,13:30,CENTRAL,NONE,300 Block of SANSOME ST,-122.401201,37.793774,"(37.793774373929594, -122.40120121358764)",17616589106374,0.0
5,176140291,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Monday,05/01/2017,22:43,TARAVAL,NONE,200 Block of VICENTE ST,-122.468724,37.739911,"(37.739910543879205, -122.46872393000805)",17614029106244,0.0
6,176140003,VANDALISM,"MALICIOUS MISCHIEF, VANDALISM",Monday,05/01/2017,19:00,TENDERLOIN,NONE,100 Block of POWELL ST,-122.408014,37.785951,"(37.78595077676134, -122.40801449292017)",17614000328150,0.0
9,176126592,LARCENY/THEFT,PETTY THEFT OF PROPERTY,Monday,05/01/2017,11:00,NORTHERN,NONE,1600 Block of JACKSON ST,-122.422253,37.794092,"(37.79409172603437, -122.4222532995645)",17612659206372,0.0
10,176122302,VANDALISM,"MALICIOUS MISCHIEF, VANDALISM",Monday,05/01/2017,07:33,BAYVIEW,NONE,1700 Block of REVERE AV,-122.392776,37.733078,"(37.73307817100114, -122.39277577233712)",17612230228150,0.0
11,176121978,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Monday,05/01/2017,22:00,BAYVIEW,NONE,1300 Block of DEHARO ST,-122.400275,37.752863,"(37.75286311521571, -122.40027509151389)",17612197806244,0.0
12,176121956,VANDALISM,"MALICIOUS MISCHIEF, VANDALISM",Monday,05/01/2017,22:00,BAYVIEW,NONE,HAMILTON ST / SILLIMAN ST,-122.411015,37.729687,"(37.72968680653694, -122.41101460446384)",17612195628150,0.0


In [12]:
crime_data.groupby(['type'], as_index = False).agg({'IncidntNum':'nunique'})\
.sort_values(by = 'IncidntNum',ascending = False)

,type,IncidntNum
0,0.0,61275
1,1.0,23398


In [13]:
crime_data.to_csv('crime_data.csv')